Update descriptions with already saved ones in description file.

Before using this notebook, make sure you have follow these steps:
1) make sure the `parameters.yml` file in the same directory.
2) Then run the code for zero timesteps with `../../run_fargo start -N 0 config.yml` inside this directory. `WriteDefaultValues: Yes` needs to be in the configfile.

In [1]:
import yaml
try:
    with open("parameters.yml", "r") as infile:
        parameters = yaml.safe_load(infile)
except FileNotFoundError:
    paramters = {}

In [2]:
parameters

{'ALPHAVISCOSITY': {'choices': '',
  'default': 0,
  'deprecated': True,
  'description': 'Specify the value of alpha for the alpha viscosity model.',
  'newname': 'ViscousAlpha'},
 'AccreteWithoutDiskFeedback': {'choices': '',
  'default': False,
  'description': 'Let planets accrete without disk feedback being enabled.',
  'type': 'bool',
  'unitsupport': False},
 'Adiabatic': {'choices': '',
  'default': False,
  'deprecated': True,
  'description': 'Deprecated. Dont use this anymore.',
  'hint': 'EquationOfState: Ideal',
  'newname': 'none',
  'type': 'string',
  'unitsupport': False},
 'AdiabaticIndex': {'choices': '',
  'default': 'none',
  'description': 'Numerical value or FIT_ISOTHERMAL (only for polytropic equation of state)',
  'type': 'double',
  'unitsupport': False},
 'AlphaCold': {'choices': '',
  'default': 0.01,
  'description': 'Alpha value for the cold state in scurve alpha.',
  'type': 'double',
  'unitsupport': False},
 'AlphaHot': {'choices': '',
  'default': 0.1,

In [3]:
defaultfile = "output/out/default_config.yml"

with open(defaultfile, "r") as infile:
    defaults = yaml.safe_load(infile)

# clean up types
for key, val in defaults.items():
    t = val["type"]
    if "basic_string" in t:
        t = "string"
    elif t == "d":
        t = "double"
    elif t == "i":
        t = "int"
    elif t == "b":
        t = "bool"
    elif t == "j":
        t = "unsigned int"
    val["type"] = t

    

In [4]:
defaults["AdiabaticIndex"]

{'default': 1.4, 'type': 'bool', 'unitsupport': False}

In [6]:
# add deprecated flag to parameters that are not in the default file
lkeys_to_remove = []
param_old_lkeys = {key.lower() : key for key in parameters}
default_lkeys = {key.lower() : key for key in defaults}
for key in parameters:
    lkey = key.lower()
    if not lkey in default_lkeys:
        lkeys_to_remove.append(key)
for key in lkeys_to_remove:
    key = param_old_lkeys[key.lower()]
    parameters[key]["deprecated"] = True

new_params = {}

for key in defaults:
    lkey = key.lower()
    # copy the old param dict to an entry with new capitalization
    if lkey in param_old_lkeys:
        oldkey = param_old_lkeys[lkey]
        new_params[key] = parameters[oldkey]


    desc = ""
    try:
        defvalue = defaults[key]["default"]
    except KeyError:
        defvalue = "none"
    # add new parameters
    if not key in new_params:
        new_params[key] = {
            "default" : defvalue,
            "description" : desc,
            "choices" : ""
        }

    # update types
    new_params[key]["type"] = defaults[key]["type"]
    # update unitsupport
    if "unitsupport" in defaults[key]:
        new_params[key]["unitsupport"] = defaults[key]["unitsupport"]
    # manually overwrite unit support for base units
    if key in ["l0", "m0", "temp0", "t0"]:
        new_params[key]["unitsupport"] = True
    # update default values
    if new_params[key]["default"] != defvalue:
        new_params[key]["default"] = defvalue

# copy left over deprecated parameters
for key, val in parameters.items():
    if "deprecated" in parameters[key]:
        new_params[key] = parameters[key]
        

In [7]:
deprecated_params = {}
for key, val in new_params.items():
    if "deprecated" in val:
        deprecated_params[key] = val
        print(key, val)

Adiabatic {'choices': '', 'default': False, 'deprecated': True, 'description': 'Deprecated. Dont use this anymore.', 'hint': 'EquationOfState: Ideal', 'newname': 'none', 'type': 'bool', 'unitsupport': False}
ALPHAVISCOSITY {'choices': '', 'default': 0, 'deprecated': True, 'description': 'Specify the value of alpha for the alpha viscosity model.', 'newname': 'ViscousAlpha'}
AlphaThreshold {'choices': 'positive values', 'default': 5000, 'deprecated': True, 'description': '?Threshold value for viscosity stabalized?', 'newname': 'none', 'type': 'double', 'unitsupport': False}
CoolingRadiativeLocal {'choices': 'yes, no', 'default': False, 'deprecated': True, 'description': 'Enable local radiative cooling Q- = 2 * sigma_R * T_eff^4.', 'hint': 'SurfaceCooling: thermal', 'newname': 'none'}
CoolingScurve {'choices': 'yes, no', 'default': False, 'deprecated': True, 'description': 'Enable Scurve Cooling', 'hint': 'SurfaceCooling: scurve', 'newname': 'none'}
DT {'choices': '', 'default': 1, 'depre

In [8]:

with open("new_parameters.yml", "w") as outfile:
    yaml.dump(new_params, outfile, width=float("inf"))

In [9]:
with open("deprecated_parameters.yml", "w") as outfile:
    yaml.dump(deprecated_params, outfile, width=float("inf"))

In [10]:
for key, val in new_params.items():
    if "oldname" in val:
        print(key, val["oldname"])

Nmonitor Ninterm
ViscousAlpha AlphaViscosity
ConstantViscosity Viscosity


In [11]:
with open("deprecated_parameters.yml", "r") as infile:
    deprecated = yaml.safe_load(infile)

for key, val in deprecated.items():
    try:
        newname = val["newname"]
    except KeyError:
        print(f"!!!!!!!!!!! '{key}' does not have 'newname' yet, if it's removed without replacement set 'newname' to 'none'")
        continue
    if newname == "none":
        if "hint" in val:
            hint = val["hint"]
            print(f"'{key}': have a look at {hint}")
        else:
            print(f"'{key}' removed without replacement")
    else:
        print(f"'{key}' renamed to '{val['newname']}'")

'ALPHAVISCOSITY' renamed to 'ViscousAlpha'
'Adiabatic': have a look at EquationOfState: Ideal
'AlphaThreshold' removed without replacement
'CoolingRadiativeLocal': have a look at SurfaceCooling: thermal
'CoolingScurve': have a look at SurfaceCooling: scurve
'DT' renamed to 'MonitorTimestep'
'DebugOutputs' removed without replacement
'DomegaDrZero': have a look at OuterBoundaryAzi = zeroshear
'ExplicitViscosity': have a look at Set ViscousAlpha > 0 or ConstantViscosity > 0
'ForcedCircular' removed without replacement
'HeatingStar': have a look at Set temperature of any Nbody object > 0 to enable irradiation from this object.
'HeatingStarFactor': have a look at Change the temperature of the Nbody object to modulate the heating.
'HeatingStarRampingTime': have a look at Use key 'irradiation ramp-up time' for Nbody objects.
'HeatingStarSimple': have a look at This is now the only option.
'IntegratePlanets' removed without replacement
'NSEC' renamed to 'Naz'
'NTOT' renamed to 'Nsnapshots'
'N

In [12]:
with open("deprecated_parameters.yml", "r") as infile:
    deprecated = yaml.safe_load(infile)

replacements = {}
for key, val in deprecated.items():
    newname = val["newname"]
    replacements[key] = {"newname": newname}
    if newname == "none":
        if "hint" in val:
            replacements[key]["hint"] = val["hint"]
replacements

{'ALPHAVISCOSITY': {'newname': 'ViscousAlpha'},
 'Adiabatic': {'newname': 'none', 'hint': 'EquationOfState: Ideal'},
 'AlphaThreshold': {'newname': 'none'},
 'CoolingRadiativeLocal': {'newname': 'none',
  'hint': 'SurfaceCooling: thermal'},
 'CoolingScurve': {'newname': 'none', 'hint': 'SurfaceCooling: scurve'},
 'DT': {'newname': 'MonitorTimestep'},
 'DebugOutputs': {'newname': 'none'},
 'DomegaDrZero': {'newname': 'none', 'hint': 'OuterBoundaryAzi = zeroshear'},
 'ExplicitViscosity': {'newname': 'none',
  'hint': 'Set ViscousAlpha > 0 or ConstantViscosity > 0'},
 'ForcedCircular': {'newname': 'none'},
 'HeatingStar': {'newname': 'none',
  'hint': 'Set temperature of any Nbody object > 0 to enable irradiation from this object.'},
 'HeatingStarFactor': {'newname': 'none',
  'hint': 'Change the temperature of the Nbody object to modulate the heating.'},
 'HeatingStarRampingTime': {'newname': 'none',
  'hint': "Use key 'irradiation ramp-up time' for Nbody objects."},
 'HeatingStarSimple'